# Problem Statement

## Part 1
* input: any number adjacent to a symbol, including diagonally, is a "part number"
* goal: sum all the part numbers

### Idea:
* Parse each number into a `PotentialPartNumber`
    <!-- * has a `is_valid` method that searches the surrounding 2d points for a symbol -->
    * has a `get_surrounding_coords` method that gets the coords of the points that surround the number and sets it to self
* Parse each symbol into a Symbol
    * has a 2d coordinate (x,y)

* symbols = {coordinate: symbol}

* _note_: using X, Y where origin is in top left
<!-- def is_valid(self):
    for surrounding_coord in surrounding_coords:
        if  -->


In [4]:
from dataclasses import dataclass
import re


@dataclass(unsafe_hash=True)
class Coordinate:
    x: int
    y: int


@dataclass
class PotentialPart:
    part_number: int
    x_start: int
    x_end: int
    y: int

    surrounding_coordinates: list[Coordinate]

    def set_surrounding_coordinates(self):
        surrounding_x_start = self.x_start - 1
        surrounding_x_end = self.x_end + 1
        surrounding_y_above = self.y - 1
        surrounding_y_below = self.y + 1

        for y in range(surrounding_y_above, surrounding_y_below + 1):
            for x in range(surrounding_x_start, surrounding_x_end + 1):
                coord = Coordinate(x, y)
                self.surrounding_coordinates.append(coord)


@dataclass
class Symbol:
    coordinate: Coordinate
    symbol_char: str


def parse_line_into_potential_parts(
    line: str, line_y: int
) -> list[PotentialPart]:
    potential_parts = []

    number_matches = re.finditer(r"\d+", line)

    for match in number_matches:
        part_number = match.group(0)
        x_start = match.span(0)[0]
        x_end = (
            match.span(0)[1] - 1
        )  # re considers span end exclusive, which isn't what I want

        potential_part = PotentialPart(
            int(part_number), x_start, x_end, line_y, surrounding_coordinates=[]
        )
        potential_part.set_surrounding_coordinates()

        potential_parts.append(potential_part)

    return potential_parts


def parse_line_into_symbols(line: str, line_y: int) -> list[Symbol]:
    symbols = []

    # capture anything that is not a digit or period, aka symbols
    symbol_regex = r"(?!\d+|\.)(.)"
    symbol_matches = re.finditer(symbol_regex, line)

    for match in symbol_matches:
        symbol_coord = Coordinate(match.span(0)[0], line_y)
        symbol_char = match.group(0)

        symbol = Symbol(symbol_coord, symbol_char)

        symbols.append(symbol)

    return symbols


def parse_lines_into_potential_parts_and_symbols(
    lines: list[str],
) -> tuple[list[PotentialPart], list[Symbol]]:
    potential_parts = []
    symbols = []

    for y, line in enumerate(lines):
        line_potential_parts = parse_line_into_potential_parts(line, y)
        potential_parts.extend(line_potential_parts)

        line_symbols = parse_line_into_symbols(line, y)
        symbols.extend(line_symbols)

    return (potential_parts, symbols)


def create_symbols_map(symbols: list[Symbol]) -> dict[Coordinate, Symbol]:
    symbols_map: dict[Coordinate, Symbol] = {}

    for symbol in symbols:
        symbols_map[symbol.coordinate] = symbol.symbol_char

    return symbols_map


def filter_valid_parts(
    potential_parts: list[PotentialPart], symbols_map: dict[Coordinate, Symbol]
) -> list[PotentialPart]:
    valid_parts = []

    for part in potential_parts:
        for coord in part.surrounding_coordinates:
            if symbols_map.get(coord) is not None:
                valid_parts.append(part)
                break

    return valid_parts

In [5]:
# example validation
schematic = """
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
"""

ex_schematic = [line for line in schematic.split("\n") if line != ""]

ex_part_numbers = [467, 35, 633, 617, 592, 755, 664, 598]

# ---

potential_parts, symbols = parse_lines_into_potential_parts_and_symbols(
    ex_schematic
)


for potential_part in potential_parts:
    print(potential_part.part_number)

print()

for symbol in symbols:
    print(symbol.symbol_char)

print()

symbols_map = create_symbols_map(symbols)
for key, value in symbols_map.items():
    print(f"{key}: {value}")

print()

valid_parts = filter_valid_parts(potential_parts, symbols_map)
for valid_part in valid_parts:
    print(valid_part.part_number)

print()

valid_part_nums = [part.part_number for part in valid_parts]
result = sum(valid_part_nums)
print(result)

467
114
35
633
617
58
592
755
664
598

*
#
*
+
$
*

Coordinate(x=3, y=1): *
Coordinate(x=6, y=3): #
Coordinate(x=3, y=4): *
Coordinate(x=5, y=5): +
Coordinate(x=3, y=8): $
Coordinate(x=5, y=8): *

467
35
633
617
592
755
664
598

4361


In [9]:
def main_part_1():
    with open("input.txt", "r") as file:
        lines = file.readlines()

    potential_parts, symbols = parse_lines_into_potential_parts_and_symbols(
        lines
    )
    symbols_map = create_symbols_map(symbols)
    valid_parts = filter_valid_parts(potential_parts, symbols_map)

    valid_part_nums = [part.part_number for part in valid_parts]
    result = sum(valid_part_nums)
    print(result)


# def main_part_2():


main_part_1()
# main_part_2()

531932
